In [202]:
from bs4 import BeautifulSoup
import pandas as pd
import requests
import re


In [203]:
apn = '7455003017'

GET request to get PIN

In [204]:
get_pin_url = f'https://zimas.lacity.org/ajaxSearchResults.aspx?search=apn&apn={apn}'
response = requests.get(get_pin_url)
text_response = response.text

print(text_response)

{type: "APN",
pin: "015B201    24",
Address: "439 W 4TH ST",
addresses: [],
selectedPins: [{pin: "015B201    24",
addresses: ["439 W 4TH ST",
"437 W 4TH ST"]},
{pin: "015B201    25",
addresses: [""]},
{pin: "015B201    44",
addresses: ["444 W 5TH ST",
"448 W 5TH ST"]},
{pin: "015B201    45",
addresses: ["438 W 5TH ST"]}]}


Exract pin from text response from Text Response

In [205]:
match = re.search(r'pin: "(.*?)"', text_response)
if match:
  pin = match.group(1)
  print(pin)
else:
  print("No pin found in the response text")

pin_parts = pin.split()
if len(pin_parts) == 2:
  pin1, pin2 = pin_parts[0], pin_parts[1]

  print(pin1)
  print(pin2)

015B201    24
015B201
24


GET request to get HTML response

In [206]:
get_html_url = f'https://zimas.lacity.org/map.aspx?pin={pin1}%20%20%20%20{pin2}&ajax=yes'
print(get_html_url)
response = requests.get(get_html_url)
html_response = response.text

print(html_response)

https://zimas.lacity.org/map.aspx?pin=015B201%20%20%20%2024&ajax=yes
{Address: "439 W 4TH ST", Addresses: ["439 W 4TH ST", "437 W 4TH ST"] , selectedPins: [{pin: '015B201    24', addresses: ["439 W 4TH ST", "437 W 4TH ST"] }], divTab1: "<table width=\"100%\" cellpadding=\"0\" cellspacing=\"0\" style=''\><tbody\><tr\><td class=\"DataCellsLeft\"\><a style='cursor: pointer;' onclick=\"getToolTip('address.htm');\"\>Site Address&nbsp;</a\></td\><td class=\"DataCellsRight\" \>439 W 4TH ST</td\></tr\><tr\><td class=\"DataCellsLeft\"\><a style='cursor: pointer;' onclick=\"getToolTip('address.htm');\"\>Site Address&nbsp;</a\></td\><td class=\"DataCellsRight\" \>437 W 4TH ST</td\></tr\><tr\><td class=\"DataCellsLeft\"\><a style='cursor: pointer;' onclick=\"getToolTip('zipcode.htm');\"\>ZIP Code&nbsp;</a\></td\><td class=\"DataCellsRight\" \><a  href=\"javascript:;\" onclick=\"ZimasData.openDataLink('ZIP_CD', '90731');\"\>90731</a\></td\></tr\><tr\><td class=\"DataCellsLeft\"\><a style='cursor: p

Convert HTML response to dataset

In [207]:
# def parse_html_table(html_content):
#     soup = BeautifulSoup(html_content, 'html.parser')
#     table = soup.find('table')

#     data = []
#     if table:
#         rows = table.find_all('tr')
#         for row in rows:
#             cells = row.find_all(['td', 'th'])
#             if len(cells) == 2:
#                 key = cells[0].get_text(strip=True).replace('\xa0', ' ')
#                 value = cells[1].get_text(strip=True).replace('\xa0', ' ')
#                 data.append((key, value))
#     return data

# def extract_data(content):
#     html_parts = re.findall(r'<table.*?</table>', content, re.DOTALL)

#     dataset = {}
#     for html_part in html_parts:
#         data = parse_html_table(html_part)
#         for key, value in data:
#             dataset[key] = value

#     return dataset

# data_dict = extract_data(html_response)

# df = pd.DataFrame(list(data_dict.items()), columns=['Field', 'Value'])
# df


In [208]:
# with open('zimas_example.html', 'r', encoding='utf-8') as file:
#     html_response = file.read()

html_response = """
<html><head><meta http-equiv=Content-Type" content="text/html; charset=windows-1252"></head><body>
"""+ html_response
html_response = html_response + "</body></html>"

content = html_response

content = content.replace("&nbsp;", " ").replace("&lt;", "<").replace("&gt;", ">")
    
# Fix tags by removing extra backslashes
content = re.sub(r"\\>", ">", content)
content = re.sub(r"\\<", "<", content)
    
# Format the HTML content with proper indentation
formatted_content = ""
indentation_level = 0
for line in content.splitlines():
  stripped_line = line.strip()
  if stripped_line.startswith("</"):
    indentation_level -= 1
  formatted_content += "    " * indentation_level + stripped_line + "\n"
  if stripped_line.startswith("<") and not stripped_line.startswith("</") and not stripped_line.endswith("/>"):
    indentation_level += 1
    


# Parse the HTML content using BeautifulSoup
soup = BeautifulSoup(content, 'html.parser')

# Find all the table rows
rows = soup.find_all('tr')

# Initialize the variable for Community Plan Area
community_plan_area_text = None

# Iterate through rows to find the "Community Plan Area" cell
for row in rows:
    cells = row.find_all('td')
    if cells and 'Community Plan Area' in cells[0].get_text(strip=True):
        community_plan_area_text = cells[1].get_text(strip=True)
        break

if community_plan_area_text:
    print("Community Plan Area:", community_plan_area_text)
else:
    print("Community Plan Area not found.")

Community Plan Area: San Pedro


In [209]:
filename = "zimas_ex.html"

# Open the file in write mode and write the data
with open(filename, "w") as file:
    file.write(content)

In [210]:
# soup = BeautifulSoup(html_response, 'html.parser')

# tables = soup.find_all('table')
# dataframes = []
# for table in tables:
#         headers = [header.text.strip() for header in table.find_all('th')]
        
#         rows = []
#         for row in table.find_all('tr'):
#             cols = row.find_all(['td', 'th'])
#             rows.append([col.text.strip() for col in cols])
        
#         df = pd.DataFrame(rows)
        
#         if headers:
#             df.columns = headers
#         else:
#             df.columns = range(len(df.columns))
        
#         dataframes.append(df)
    


# for i, df in enumerate(dataframes):
#     print(f"Table {i+1}:")
#     print(df)
#     print("\n")
